# ISM Benchmark

Benchmark NaiveISM and fastISM.

In [1]:
import sys
sys.path.append("../")

import fastISM
from fastISM.models.basset import basset_model
from fastISM.models.factorized_basset import factorized_basset_model
from fastISM.models.bpnet import bpnet_model
import tensorflow as tf
import numpy as np
from importlib import reload
import time

In [2]:
tf.__version__

'2.3.0'

In [3]:
!nvidia-smi

Tue Jan 25 16:42:22 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  On   | 00000000:03:00.0 Off |                    0 |
| N/A   33C    P0    30W / 250W |      0MiB / 16280MiB |      0%   E. Process |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-27db8534-9b2b-8b1a-5889-9c77c0c7be4e)


In [5]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Wed_Apr_24_19:10:27_PDT_2019
Cuda compilation tools, release 10.1, V10.1.168


In [6]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [7]:
device = 'GPU:0' if tf.config.experimental.list_physical_devices('GPU') else '/device:CPU:0'
device

'GPU:0'

In [8]:
seqs = np.load("test.seq.npy")

In [9]:
seqs = np.vstack([seqs, seqs])

In [10]:
seqs.shape

(10000, 2000, 4)

## Benchmark

Best practice would be to restart kernel after benchmarking each model!

In [11]:
def time_ism(ism_model, batch_sizes, seqlen):
    times = []
    per_100 = []
    for b in batch_sizes:

        # dry run -- required as first batch slower for setting up
        #            and variable batch sizes (due to varying number
        #            of seqs that need to be mutated at a position) 
        #            also slows down first call
        # x = np.random.random((b,seqlen,4))
        x = seqs[:b, :seqlen]        
        x = tf.constant(x, dtype=ism_model.model.inputs[0].dtype)
        o = ism_model(x, [0,0,0,1])
        
        t = time.time()
        x = tf.constant(x, dtype=ism_model.model.inputs[0].dtype)
        
        # NOTE: computations are only performed at those positions
        # at which the existing base != replace_with
        o = ism_model(x, replace_with=[0,0,0,1])
        o = ism_model(x, replace_with=[0,0,1,0])
        o = ism_model(x, replace_with=[0,1,0,0])
        o = ism_model(x, replace_with=[1,0,0,0])
        
        times.append(time.time()-t)
        
        per_100.append((times[-1]/b)*100)
        print("BATCH: {}\tTIME: {:.2f}\tPER 100: {:.2f}".format(b, times[-1], (times[-1]/b)*100))
    
    print("BEST PER 100: {:.2f}".format(min(per_100)))

### Basset (1000)

In [12]:
model = basset_model(seqlen=1000, num_outputs=1)

In [13]:
model_fism = fastISM.FastISM(model, test_correctness=False, change_ranges=[(x,x+1) for x in range(550,650)])

In [14]:
time_ism(model_fism, [4310], 1000)

BATCH: 4310	TIME: 22.82	PER 100: 0.53
BEST PER 100: 0.53


In [13]:
model_nism = fastISM.NaiveISM(model, change_ranges=[(x,x+1) for x in range(595,600)])

In [ ]:
time_ism(model_nism, [4320], 1000)

### Basset (2000)

In [12]:
model = basset_model(seqlen=2000, num_outputs=1)

In [13]:
model_fism = fastISM.FastISM(model, test_correctness=False, change_ranges=[(x,x+1) for x in range(975,1025)])

In [ ]:
time_ism(model_fism, [2140], 2000)

In [13]:
model_nism = fastISM.NaiveISM(model, change_ranges=[(x,x+1) for x in range(995,1000)])

In [ ]:
time_ism(model_nism, [2150], 2000)

### Factorized Basset (1000)

In [12]:
model = factorized_basset_model(seqlen=1000, num_outputs=1)

In [13]:
model_fism = fastISM.FastISM(model, test_correctness=False, change_ranges=[(x,x+1) for x in range(575,625)])

In [14]:
time_ism(model_fism, [3072], 1000)

BATCH: 3072	TIME: 26.76	PER 100: 0.87
BEST PER 100: 0.87


In [15]:
model_nism = fastISM.NaiveISM(model, change_ranges=[(x,x+1) for x in range(575,625)])

In [16]:
time_ism(model_nism, [3700], 1000)

BATCH: 3700	TIME: 274.45	PER 100: 7.42
BEST PER 100: 7.42


### Factorized Basset (2000)

In [12]:
model = factorized_basset_model(seqlen=2000, num_outputs=1)

In [13]:
model_fism = fastISM.FastISM(model, test_correctness=False, change_ranges=[(x,x+1) for x in range(975,1025)])

In [14]:
time_ism(model_fism, [1530], 2000)

BATCH: 1530	TIME: 17.50	PER 100: 1.14
BEST PER 100: 1.14


In [16]:
model_nism = fastISM.NaiveISM(model, change_ranges=[(x,x+1) for x in range(975,1025)])

In [17]:
# FINAL
time_ism(model_nism, [1840], 2000)

BATCH: 1840	TIME: 166.70	PER 100: 9.06
BEST PER 100: 9.06


### BPNet (1000)

In [12]:
model = bpnet_model(seqlen=1000, num_dilated_convs=9)

In [13]:
model_fism = fastISM.FastISM(model, test_correctness=False, change_ranges=[(x,x+1) for x in range(575,625)])

In [15]:
# FINAL
time_ism(model_fism, [1360], 1000)

BATCH: 1360	TIME: 23.56	PER 100: 1.73
BEST PER 100: 1.73


In [14]:
model_nism = fastISM.NaiveISM(model, change_ranges=[(x,x+1) for x in range(575,625)])

In [15]:
# FINAL
time_ism(model_nism, [8510], 1000)

BATCH: 8510	TIME: 351.94	PER 100: 4.14
BEST PER 100: 4.14


### BPNet (2000)

In [12]:
model = bpnet_model(seqlen=2000, num_dilated_convs=9)

In [13]:
model_fism = fastISM.FastISM(model, test_correctness=False, change_ranges=[(x,x+1) for x in range(575,625)])

In [14]:
# FINAL
time_ism(model_fism, [800], 2000)

BATCH: 800	TIME: 22.11	PER 100: 2.76
BEST PER 100: 2.76


In [15]:
model_nism = fastISM.NaiveISM(model, change_ranges=[(x,x+1) for x in range(575,625)])

In [16]:
# FINAL
time_ism(model_nism, [4550], 2000)

BATCH: 4550	TIME: 331.98	PER 100: 7.30
BEST PER 100: 7.30
